In [1]:
#https://github.com/garywu/google-compute-engine-selenium
#install chromium, its driver, and selenium
# !sudo apt install chromium-chromedriver
# !sudo cp /usr/lib/chromium-browser/chromedriver /usr/bin
# !pip install selenium
# set options to be headless, ..
import urllib.request
import csv

import time
import random


from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver= webdriver.Chrome('chromedriver',options=options)


# open it, go to a website, and get results
# wd = webdriver.Chrome('chromedriver',options=options)
# wd.get("https://www.website.com")
# print(wd.page_source)  # results

In [2]:
import csv


f = open("polarnopyret.csv", "w", encoding='utf-8')
writer = csv.DictWriter(f, fieldnames=["name","brand","gender","category","size","price","composition","color","url","img_url","article_no"])
writer.writeheader()
f.close()

import pandas as pd
df=pd.read_csv("polarnopyret.csv", encoding='utf-8')
df.head()

,name,brand,gender,category,size,price,composition,color,url,img_url,article_no


In [3]:
!wget -O "auth.json" https://dl.dropbox.com/s/p1ifzzf2443r05g/urbankiddo-c8528ce11ed0.json

--2019-07-25 14:23:43--  https://dl.dropbox.com/s/p1ifzzf2443r05g/urbankiddo-c8528ce11ed0.json
Resolving dl.dropbox.com (dl.dropbox.com)... 162.125.1.6, 2620:100:601b:6::a27d:806
Connecting to dl.dropbox.com (dl.dropbox.com)|162.125.1.6|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://dl.dropboxusercontent.com/s/p1ifzzf2443r05g/urbankiddo-c8528ce11ed0.json [following]
--2019-07-25 14:23:43--  https://dl.dropboxusercontent.com/s/p1ifzzf2443r05g/urbankiddo-c8528ce11ed0.json
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.1.6, 2620:100:6016:6::a27d:106
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.1.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2315 (2.3K) [text/plain]
Saving to: ‘auth.json’

auth.json           100%[===================>]   2.26K  --.-KB/s    in 0s      

2019-07-25 14:23:43 (39.2 MB/s) - ‘auth.json’ saved [2315/2315]



In [4]:
from google.cloud import storage

def upload_to_bucket(blob_name, path_to_file):
    """ Upload data to a bucket""" #doc string in function

    # Explicitly use service account credentials by specifying the private key
    # file.
    storage_client = storage.Client.from_service_account_json(
        'auth.json')

    #print(buckets = list(storage_client.list_buckets())

    bucket = storage_client.get_bucket('urbankiddo')
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(path_to_file)

    #returns a public url
    return blob.public_url

In [5]:
!mkdir polarnopyret

mkdir: cannot create directory ‘polarnopyret’: File exists


In [6]:
import urllib.request
import csv

import time
import random
error=[]
def scrapper(main_url,gender):
    driver= webdriver.Chrome('chromedriver',options=options)
    driver.get(main_url)
    error=[]     
    for c in range(5,59):        
        driver= webdriver.Chrome('chromedriver',options=options)
        driver.implicitly_wait(10) # seconds
        driver.get(main_url)
        driver.set_window_size(764,784)

        xpath='//*[@id="container"]/div/div/div[1]/main/div[2]/div[2]/div[3]/div[1]/div[1]/button'
        filter = driver.find_element_by_xpath(xpath)
        driver.execute_script("(arguments[0]).click();", filter)

        xpath2='//*[@id="container"]/div/div/div[1]/main/div[2]/div[3]/div[2]/div/div/div[2]/div/div[2]/div[1]/button'
        kategorier= driver.find_element_by_xpath(xpath2)
        driver.execute_script("(arguments[0]).click();", kategorier)

        xpath3='//*[@id="container"]/div/div/div[1]/main/div[2]/div[3]/div[2]/div/div/div[2]/div/div[2]/div[2]/div/div/div['+str(c)+']/label/input'
        checkbox= driver.find_element_by_xpath(xpath3)
        driver.execute_script("(arguments[0]).click();", checkbox)

        xpath4='//*[@id="container"]/div/div/div[1]/main/div[2]/div[3]/div[2]/div/div/div[2]/div/div[2]/div[2]/div/div/div['+str(c)+']/label/span'
        category=driver.find_element_by_xpath(xpath4).text  ## category
        print(category)

        xpath5='//*[@id="container"]/div/div/div[1]/main/div[2]/div[3]/div[2]/div/div/div[3]/button[2]'
        view=driver.find_element_by_xpath(xpath5)
        driver.execute_script("(arguments[0]).click();", view)

        xpath6='//*[@id="container"]/div/div/div[1]/main/div[2]/div[2]/div[3]/div[3]/button'
        load_more=driver.find_element_by_xpath(xpath6)
        driver.execute_script("(arguments[0]).click();", load_more)

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        time.sleep(30)

        products=driver.find_elements_by_css_selector("div[class^='sg-product-card']")
        print(len(products))


        urls=[]
        for p in products:
            urls.append(p.find_element_by_tag_name('a').get_attribute('href'))
        driver.quit()


        for u in urls:

            try:
                url=u
                driver= webdriver.Chrome('chromedriver',options=options)
                driver.get(url)
                driver.set_window_size(764,784)
                name=driver.find_element_by_tag_name('h1').text
                article_no=driver.find_element_by_xpath('//*[@id="container"]/div/div/div[1]/main/div[2]/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div/div[3]/p[3]/span').text
                price=driver.find_element_by_xpath('//*[@id="container"]/div/div/div[1]/main/div[2]/div[2]/div[2]/div[1]/div[1]/div/div/div[2]/span').text
                price
                brand='polarnopyret'
                composition=driver.find_element_by_xpath('//*[@id="container"]/div/div/div[1]/main/div[2]/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div/div[3]/p[2]/span').text
                color=driver.find_element_by_xpath('//*[@id="container"]/div/div/div[1]/main/div[2]/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div/div[3]/p[1]/span').text
                img_url=driver.find_element_by_xpath('//*[@id="container"]/div/div/div[1]/main/div[2]/div[2]/div[1]/div/div/div[1]/div[1]/div/div/div[1]/div/img').get_attribute('src')
                #             img_url = "https:" + img_url
                img_url=img_url.replace(' ','+')
                ac='polarnopyret'+'/'+article_no+str('.jpg')
                print(img_url)
                opener = urllib.request.URLopener()
                opener.addheader('User-Agent', 'whatever')
                filename, headers2 = opener.retrieve(img_url, ac)

#                 blob_name=str('polarnopyret')+str('/')+str(article_no)
#                 path_to_file='polarnopyret/'+article_no+str('.jpg')
#                 blob_url=upload_to_bucket(blob_name, path_to_file)


                sizes=driver.find_elements_by_xpath('//*[@id="container"]/div/div/div[1]/main/div[2]/div[2]/div[2]/div[1]/div[2]/div[2]/div')
                for s in sizes:
                    
                    size=s.text
#                     print(size,article_no,brand,name,color,url,price)

                with open("polarnopyret.csv", "a", encoding='utf-8') as f:
                    
                    writer = csv.writer(f)
                    writer.writerow([name,brand,gender,category,size,price,composition,color,url,img_url,article_no])
                driver.quit()
            except:
                error.append(url)
                driver.quit()
                continue



In [7]:
scrapper(main_url='https://www.polarnopyret.se/barnklader?sortBy=popularity&count=30',gender='unisex')



Underkläder & Nattplagg
(292)
292
https://www.polarnopyret.se/globalassets/productimages-polarnopyret/7325853830547.jpg?ref=E23BD844B6&w=960&h=960&mode=max


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.polarnopyret.se/globalassets/productimages-polarnopyret/7325853668157.jpg?ref=37CAD4411F&w=960&h=960&mode=max


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.polarnopyret.se/globalassets/productimages-polarnopyret/7325853750494.jpg?ref=4E4EEF21B8&w=960&h=960&mode=max


...

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.polarnopyret.se/globalassets/productimages-polarnopyret/7325853569959.jpg?ref=A8CC2C5D3A&w=960&h=960&mode=max


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


Jackor
(46)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="container"]/div/div/div[1]/main/div[2]/div[2]/div[3]/div[3]/button"}
  (Session info: headless chrome=75.0.3770.142)
  (Driver info: chromedriver=2.36.540471 (9c759b81a907e70363c6312294d30b6ccccc2752),platform=Linux 4.9.0-9-amd64 x86_64)


In [8]:
from google.cloud import storage
storage_client = storage.Client.from_service_account_json('auth.json')
bucket = storage_client.get_bucket('urbankiddo')
blob = bucket.blob('polarnopyret.csv')
blob.upload_from_filename('polarnopyret.csv')
blob.public_url

'https://storage.googleapis.com/urbankiddo/polarnopyret.csv'